In [ ]:
############################################################################
############################################################################
# 日経一目均衡表その2
import numpy as np
import pandas as pd
import talib as ta


def initialize(ctx):
    # 設定
    ctx.logger.debug("initialize() called")
    ctx.configure(
      channels={          # 利用チャンネル
        "jp.stock": {
          "symbols": [
             "jp.stock.9983", #ファーストリテイリング
            "jp.stock.9984", #ソフトバンク
            "jp.stock.6954", #ファナック
            "jp.stock.9433", #KDDI
            "jp.stock.8028", #ファミリーマート
            "jp.stock.8035", #東京エレクトロン
            "jp.stock.4543", #テルモ
            "jp.stock.6367", #ダイキン
            "jp.stock.6971", #京セラ
            "jp.stock.9735", #セコム
          ],
          "columns": [
           #"open_price_adj",    # 始値(株式分割調整後)
            "high_price_adj",    # 高値(株式分割調整後)
            "low_price_adj",     # 安値(株式分割調整後)
            #"volume_adj",         # 出来高
            #"txn_volume",         # 売買代金
            "close_price",        # 終値
            "close_price_adj",    # 終値(株式分割調整後) 
          ]
        }
      }
    )
    
    def _my_ichimoku(data):
      
      cp=data["close_price_adj"].fillna(method="ffill")
      hp=data["high_price_adj"].fillna(method="ffill")
      
      result1=pd.DataFrame(data=0,columns=[],index=cp.index)
      sample1=pd.DataFrame(data=0,columns=[],index=cp.index)
      result1=pd.DataFrame(data=0,columns=[],index=cp.index)
      sample3=pd.DataFrame(data=0,columns=[],index=cp.index)
      index3=pd.DataFrame(data=0,columns=[],index=cp.index)
      index6=pd.DataFrame(data=0,columns=[],index=cp.index)
      index7=pd.DataFrame(data=0,columns=[],index=cp.index)
      index8=pd.DataFrame(data=0,columns=[],index=cp.index)
      index12=pd.DataFrame(data=0,columns=[],index=cp.index)
      index13=pd.DataFrame(data=0,columns=[],index=cp.index)
      index14=pd.DataFrame(data=0,columns=[],index=cp.index)
      index15=pd.DataFrame(data=0,columns=[],index=cp.index)
      index16=pd.DataFrame(data=0,columns=[],index=cp.index)
      index17=pd.DataFrame(data=0,columns=[],index=cp.index)
      index18=pd.DataFrame(data=0,columns=[],index=cp.index)
      index19=pd.DataFrame(data=0,columns=[],index=cp.index)
      index20=pd.DataFrame(data=0,columns=[],index=cp.index)
      index21=pd.DataFrame(data=0,columns=[],index=cp.index)
      
      
      index1=data["high_price_adj"].fillna(method='ffill').rolling(window=9,center=False).max()
      index2=data["low_price_adj"].fillna(method="ffill").rolling(window=9,center=False).min()
      index3=(index1+index2)/2 #転換線
      index4=data["high_price_adj"].fillna(method='ffill').rolling(window=26,center=False).max()
      index5=data["low_price_adj"].fillna(method='ffill').rolling(window=26,center=False).min()
      index6=(index4+index5)/2 #基準線
      index7=(index3+index6)/2 
      index8=index7.shift(26) #先行スパン１
      index9=data["high_price_adj"].fillna(method='ffill').rolling(window=52,center=False).max()
      index10=data["low_price_adj"].fillna(method='ffill').rolling(window=52,center=False).min()
      index11=(index8+index9)/2 
      index12=index11.shift(26) #先行スパン2
      index13=index3-index6
      index14=index13.shift(1)
      index15=data["close_price_adj"].fillna(method='ffill').shift(26)
      index16=np.maximum(index8,index12)
      index17=np.minimum(index8,index12)
      index18=cp-index17
      index19=index18.shift()
      index20=cp-index16
      index21=index20.shift()
      
      buy_sig=index18[(index18>0)&(index19<0)]
      sell_sig=index20[(index20<0)&(index21>0)]
     
      return {
          "buy:sig":buy_sig,
          "sell:sig":sell_sig,
          "転換線:g2":index3,
          "基準線:g2":index6,
          "先行スパン1:g2":index8,
          "先行スパン2:g2":index12,
          "遅行線:g2":index15
        }

    # シグナル登録
    ctx.regist_signal("my_ichimoku", _my_ichimoku)

def handle_signals(ctx, date, current):
    
    buy = current["buy:sig"].dropna()
    for (sym,val) in buy.items():

        sec = ctx.getSecurity(sym)
        sec.order(sec.unit() * 1, comment="SIGNAL BUY")
        ctx.logger.debug("BUY: %s,  %f" % (sec.code(), val))
        pass

    sell = current["sell:sig"].dropna()
    for (sym,val) in sell.items():

        sec = ctx.getSecurity(sym)
        sec.order(sec.unit() * -1, comment="SIGNAL SELL")
        ctx.logger.debug("SELL: %s,  %f" % (sec.code(), val))
        pass

